In [51]:
import pandas as pd
def split_x(x,n=20):
    results = []
    
    # case1
    if '+1)' in x:
        for i in range(1,n):
            text = x[x.find(f'+{i})'):x.find(f'+{i+1})')]
            if len(text) != 0:
                results.append(str(text))
    
    # case2
    elif '1.' in x:
        for i in range(1,n):
            text = x[x.find(f'{i}.'):x.find(f'{i+1}.')]
            if len(text) != 0:
                results.append(str(text))
    
    return results

In [52]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['46A','47A']]
df

,46A,47A
3,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...
4,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...
5,1.SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS AN...,+ALL DOCUMENTS MUST MADE IN ENGLISH.+ALL DOCUM...
7,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...
9,+SIGNED COMMERCIAL INVOICE IN TRIPLICATE INDIC...,ALL DRAFT(S) DRAWN HEREUNDER MUST BE MARKED ''...
...,...,...
8403,1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS IND...,"1. ALL DOCUMENTS MUST BE DATED, SIGNED AND ISS..."
8413,+ COMMERCIAL INVOICE IN 3 ORIGINAL\r\n+ 2/3 SE...,+ T.T.REIMBURSEMENT : PROHIBITED\r\n+ INSURANC...
8414,+SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 OR...,+KINDLY ASSIST US TO RELAY THIS CREDIT TO CITI...
8415,+SIGNED AND STAMPED COMMERCIAL INVOICE IN 3 OR...,+KINDLY ASSIST US TO RELAY THIS CREDIT TO CITI...


In [53]:
x = df['46A'].values[172]
x

"+1) SIGNED COMMERCIAL INVOICES IN TRIPLICATE CERTIFYING\r\nMERCHANDISE ARE OF TAIWAN ORIGIN INDICATING H.S. CODE NO.\r\n3903.3000 AND ALSO CERTIFY THAT GOODS SHIPPED ARE AS PER\r\nBENEFICIARY'S PROFORMA INVOICE NO. B2018 DATED: 29-JAN-2021 AND\r\nAPPLICANT'S PURCHASE ORDER NO.TCL-L-2021-02-13520 DATED:\r\n02-FEB-2021\r\n+2) FULL SET OF ORIGINAL CLEAN 'LADEN ON BOARD' MARINE BILL\r\nOF LADING MADE OUT OR ENDORSED TO THE ORDER OF MCB BANK LIMITED,\r\nCORPORATE MAIN BOULEVARD GULBERG BRANCH, LAHORE-PAKISTAN\r\nSHOWING FREIGHT PREPAID BEARING THIS L/C NUMBER AND\r\nMARKED NOTIFY TO\r\nA) TREET CORPORATION LIMITED\r\n72-B, KOTLAKHPAT INDUSTRIAL AREA, LAHORE-PAKISTAN.\r\nB) TREET CORPORATION LIMITED\r\n6-B (A1) SAEED HAI ROAD MUHAMMAD ALI COOPERATIVE HOUSING\r\nSOCIETY, KARACHI.\r\nC) MCB BANK LIMITED\r\nCORPORATE MAIN BOULEVARD GULBERG BRANCH,\r\nLAHORE-PAKISTAN.\r\n+3) PACKING LIST IN TRIPLICATE.\r\n+4)ALL SHIPMENT DETAILS TO BE SENT WITHIN FIVE WORKING DAYS\r\nFROM SHIPMENT TO APPLICANT 

In [54]:
for i in split_x(x):
    print(i)
    print('-----------')

+1) SIGNED COMMERCIAL INVOICES IN TRIPLICATE CERTIFYING
MERCHANDISE ARE OF TAIWAN ORIGIN INDICATING H.S. CODE NO.
3903.3000 AND ALSO CERTIFY THAT GOODS SHIPPED ARE AS PER
BENEFICIARY'S PROFORMA INVOICE NO. B2018 DATED: 29-JAN-2021 AND
APPLICANT'S PURCHASE ORDER NO.TCL-L-2021-02-13520 DATED:
02-FEB-2021

-----------
+2) FULL SET OF ORIGINAL CLEAN 'LADEN ON BOARD' MARINE BILL
OF LADING MADE OUT OR ENDORSED TO THE ORDER OF MCB BANK LIMITED,
CORPORATE MAIN BOULEVARD GULBERG BRANCH, LAHORE-PAKISTAN
SHOWING FREIGHT PREPAID BEARING THIS L/C NUMBER AND
MARKED NOTIFY TO
A) TREET CORPORATION LIMITED
72-B, KOTLAKHPAT INDUSTRIAL AREA, LAHORE-PAKISTAN.
B) TREET CORPORATION LIMITED
6-B (A1) SAEED HAI ROAD MUHAMMAD ALI COOPERATIVE HOUSING
SOCIETY, KARACHI.
C) MCB BANK LIMITED
CORPORATE MAIN BOULEVARD GULBERG BRANCH,
LAHORE-PAKISTAN.

-----------
+3) PACKING LIST IN TRIPLICATE.

-----------
+4)ALL SHIPMENT DETAILS TO BE SENT WITHIN FIVE WORKING DAYS
FROM SHIPMENT TO APPLICANT THROUGH EMAIL AT
''IMPORT

In [55]:
x = df['46A'].values[0]
for i in split_x(x):
    print(i)
    print('-----------')

1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND 3 COPIES.
-----------
2. FULL SET OF SIGNED 'CLEAN ON BOARD' OCEAN BILLS OF LADINGMADE OUT TO ORDER OF LETTER OF CREDIT ISSUING BANK MARKEDFREIGHT PREPAID EVIDENCING SHIPMENT OF MERCHANDISE DESCRIBEDABOVE. BILLS OF LADING MUST STATE FULL NAME AND ADDRESS OF BOTHAPPLICANT AND ICICI BANK LTD, SHALIMAR TOWER, 31/54 M.G.MARG,HAZRATGANJ, LUCKNOW 226001,UTTAR PRADESH, INDIA AS PARTIES TOBE NOTIFIED. SHORT FORM,FREIGHT FORWARDERS AND LASHBILLS OF LADING ARE NOT ACCEPTABLE.14 DAYS FREE DETENTION TIMESHOULD BE GIVEN AT DISCHARGE PORT AND THE SAME SHOULD BEMENTIONED EITHER ON THE BILL OF LADING OR BY A SEPARATECERTIFICATE FROM SHIPPING COMPANY OR FROM THEIR AGENTS MUSTACCOMPANY ORIGINAL DOCUMENTS.
-----------
3.MARINE INSURANCE POLICY OR CERTIFICATE (1 ORIGINAL PLUS 1COPY) DATED NOT LATER THAN THE DATE OF BILLS OF LADING SIGNEDAND ISSUED BY INSURANCE COMPANY MADE TO ORDER AND BLANKENDORSED FOR 110 PCT OF CIF VALUE OF GOODS SUPPLIED, COVERINGINSTITU

In [56]:
x = df['47A'].values[0]
for i in split_x(x):
    print(i)
    print('-----------')

1. ALL DOCUMENTS MUST BE IN ENGLISH.
-----------
2. ALL DOCUMENTS MUST MENTION OUR LC NUMBER AND DATE.
-----------
3. DISCREPANCY FEE OF USD 100/- PER SET OF DISCREPANT DOCUMENTSPRESENTED WILL BE DEDUCTED FROM BILL PROCEEDS .
-----------
4. NEGOTIATING BANK MUST SEND ORIGINAL SET OF DOCUMENTS IN ONELOT TO THE FOLLOWING ADDRESS BY COURIER:ICICI BANK LTD, SHALIMAR TOWER, 31/54 M.G.MARG, HAZRATGANJ,LUCKNOW 226001,UTTAR PRADESH, INDIA
-----------
5. NOTWITHSTANDING THE CONTENT OF ARTICLE 16 OF UCP 600 , INTHE EVENT THAT WE OBSERVE DISCREPANCIES IN DOCUMENTS WE SHALLPROVIDE OUR NOTICE OF REFUSAL IN ACCORDANCE THEREWITH AND IF WEELECT TO CONTACT THE APPLICANT FOR A WAIVER, AND WE RECEIVE ANACCEPTABLE WAIVER, WE SHALL EFFECT SETTLEMENT ACCORDING TO THELC TERMS AND THE DOCUMENTS WILL BE RELEASED WITHOUT FURTHERNOTICE TO YOU, UNLESS WE ARE IN RECEIPT OF YOUR INSTRUCTION TOTHE CONTRARY PRIOR TO SUCH RELEASE.
-----------
6. THIS LC WOULD BE SUBJECT TO US, EU, UN, INDIA AND/OR OTHERSANCTIONS (INCL

In [57]:
x = df['46A'].values[5]
x

'+1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S) INDICATINGTHIS L/C NO. AND CONTRACT NO.2.PACKING LIST/WEIGHT MEMO IN 3 ORIGINAL(S) INDICATINGQUANTITY/GROSS AND NET WEIGHTS OF EACH PACKAGE AND PACKINGCONDITIONS AS CALLED FOR BY THE L/C.3.CERTIFICATE OF WOOD PACKING MATERIAL OR CERTIFICATE OF NON-WOODPACKING MATERIAL IN 2 ORIGINALS ISSUED BY BENEFICIARY.4.BL REQUEST AS BELOWFOR ITEM 1:FULL (3/3) SET OF ORIGINAL(S) OF CLEAN SHIPPED ON BOARD OCEANBILL OF LADING MADE OUT TO ORDER OFVIETINBANK BA DINH BRANCH,MARKED FREIGHT PREPAID, NOTIFY PLASTICCHEMICALS JOINT STOCK COMPANY (PLASCHEM). ADD: NO. 562 NGUYEN VANCU STR., LONG BIEN DIST,HA NOI, VIETNAM,BL SHOW HSCODE:3901,SHOWING THE NAME, ADDRESS AND TELEPHONE NO. OF THEAGENT OF SHIPPING LINE IN VIETNAM, BL NOT INDICATING THIS LC NO.,CONTRACT NO., UNIT PRICE AND PRICE TERM.FOR ITEM 2: FULL (3/3) SET OF ORIGINAL(S) OF CLEAN SHIPPED ONBOARD OCEAN BILL OF LADING MADE OUT TO ORDER AND BLANK ENDORSED,MARKED FREIGHT PREPAID, NOTIFY HUAKANG PACKAGING 

In [58]:
for i in split_x(x):
    print(i)

1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINAL (S) INDICATINGTHIS L/C NO. AND CONTRACT NO.
2.PACKING LIST/WEIGHT MEMO IN 3 ORIGINAL(S) INDICATINGQUANTITY/GROSS AND NET WEIGHTS OF EACH PACKAGE AND PACKINGCONDITIONS AS CALLED FOR BY THE L/C.
3.CERTIFICATE OF WOOD PACKING MATERIAL OR CERTIFICATE OF NON-WOODPACKING MATERIAL IN 2 ORIGINALS ISSUED BY BENEFICIARY.
4.BL REQUEST AS BELOWFOR ITEM 1:FULL (3/3) SET OF ORIGINAL(S) OF CLEAN SHIPPED ON BOARD OCEANBILL OF LADING MADE OUT TO ORDER OFVIETINBANK BA DINH BRANCH,MARKED FREIGHT PREPAID, NOTIFY PLASTICCHEMICALS JOINT STOCK COMPANY (PLASCHEM). ADD: NO. 562 NGUYEN VANCU STR., LONG BIEN DIST,HA NOI, VIETNAM,BL SHOW HSCODE:3901,SHOWING THE NAME, ADDRESS AND TELEPHONE NO. OF THEAGENT OF SHIPPING LINE IN VIETNAM, BL NOT INDICATING THIS LC NO.,CONTRACT NO., UNIT PRICE AND PRICE TERM.FOR ITEM 2: FULL (3/3) SET OF ORIGINAL(S) OF CLEAN SHIPPED ONBOARD OCEAN BILL OF LADING MADE OUT TO ORDER AND BLANK ENDORSED,MARKED FREIGHT PREPAID, NOTIFY HUAKANG PACKAGING